# Загрузка Pandas и очистка данных

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import pandas_profiling
from itertools import combinations
from scipy.stats import ttest_ind
from collections import Counter
import re

import matplotlib.pyplot as plt
%matplotlib inline

%matplotlib inline
sns.set(style="whitegrid")

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

In [2]:
main_task = pd.read_csv('main_task.csv')

In [3]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [4]:
# Переименуем названия колонок
main_task.columns = ['Restaurant_id', 'City', 'Cuisine_Style', 'Ranking', 'Rating',
                     'Price_Range', 'Number_of_Reviews', 'Reviews', 'URL_TA', 'ID_TA']

In [5]:
# Сразу удалим колонки с ID ресторанов в файле и на сайте TA
main_task = main_task.drop(['Restaurant_id', 'ID_TA'], axis=1)

In [6]:
# создадим Dummy-переменные
main_task = pd.concat([main_task, pd.get_dummies(main_task['City'])], axis=1)


price_level = {'$': 'low', '$$ - $$$': 'middle', '$$$$': 'high'}
main_task.Price_Range = main_task.Price_Range.replace(to_replace=price_level)
main_task = pd.concat(
    [main_task, pd.get_dummies(main_task['Price_Range'])], axis=1)

In [7]:
# И сразу удалим признак Price_Range т.к. модель его не примет
main_task = main_task.drop(['Price_Range'], axis=1)

In [8]:
# Создадим признаки потипу кухонь
for i in main_task.Cuisine_Style.index:
    main_task.Cuisine_Style[i] = [element.strip(
        "'[]") for element in str(main_task.Cuisine_Style[i]).split(", ")]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
Cuisine = set()  # создаём пустое множество для хранения уникальных значений стилей кухонь
for c in main_task.Cuisine_Style:  # начинаем перебор всех стилей кухонь
    for i in c:  # начинаем перебор всех стилей кухонь
        # добавляем название стиля кухни к множеству
        Cuisine.add(i)
len(Cuisine)

126

In [10]:
def find_item(cell):
    if item in cell:
        return 1
    return 0

In [11]:
for item in Cuisine:
    main_task[item] = main_task['Cuisine_Style'].apply(find_item)

In [12]:
main_task['Cuisine_Style'] = main_task['Cuisine_Style'].apply(lambda x: len(x))

In [13]:
# Приведем столбец Number_of_Reviews и Ranking к типу данных int
main_task.Ranking = main_task.Ranking.astype('int')

In [14]:
main_task['Number_of_Reviews'] = main_task['Number_of_Reviews'].fillna(0)

In [15]:
main_task.Number_of_Reviews = main_task.Number_of_Reviews.astype('int')

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [27]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = main_task.drop(['Rating', 'City', 'Reviews', 'URL_TA'], axis = 1)
y = main_task['Rating']

In [28]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [29]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [30]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [31]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [32]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.214256
